# Chapter 19. Concurrency Models in Python

In [1]:
import itertools
import time
from threading import Thread, Event


In [2]:
def spin(msg: str, done: Event) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        if done.wait(.1):
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')

def slow() -> int:
    time.sleep(3)
    return 42

In [3]:
def supervisor() -> int:
    done=Event()
    spinner = Thread(target=spin, args=('thinking!', done))
    print(f'spinner object: {spinner}')
    spinner.start()
    result = slow()
    done.set()
    spinner.join()
    return result


In [7]:
result = supervisor()
print(result)

spinner object: <Thread(Thread-5 (spin), initial)>
42          


In [5]:
from multiprocessing import Process, Event, synchronize

In [6]:
def spin(msg: str, done: synchronize.Event) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        if done.wait(.1):
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')

In [8]:
result = supervisor()
print(result)

spinner object: <Thread(Thread-6 (spin), initial)>
42          


In [10]:
import asyncio

In [12]:
async def slow() -> int:
    time.sleep(3)
    return 42

In [13]:
slow()

<coroutine object slow at 0x7fa04d6ae960>

In [14]:
async def supervisor() -> int:
    spinner = asyncio.create_task(spin('thinking'))
    result = await slow()
    spinner.cancel()
    return result

supervisor()

<coroutine object supervisor at 0x7fa04d6af3e0>